In [ ]:
info_theory = {"camb": {"extra_args": {"lens_potential_accuracy": 1}}}
#info_theory = {"classy": {"extra_args": {"non linear": "halofit"}}}

modules = ""  # SET!!!

n = 500

In [ ]:
from __future__ import division, print_function, absolute_import
from collections import OrderedDict as odict

from cobaya.conventions import _likelihood, _theory, _timing, _path_install
from cobaya.tools import recursive_update
from cobaya.cosmo_input import create_input, planck_base_model
from cobaya.model import get_model

from test_cosmo_planck_2015 import params_lowTEB_highTTTEEE

info = create_input(planck_names=True, theory=list(info_theory)[0], **planck_base_model)
info = recursive_update(info, {_theory: info_theory})

likelihoods = [
    "planck_2015_lowl", "planck_2015_lowTEB",
    "planck_2015_plikHM_TT", "planck_2015_plikHM_TTTEEE",
    #"planck_2015_plikHM_TT_unbinned", "planck_2015_plikHM_TTTEEE_unbinned",
    "planck_2015_lensing", "planck_2015_lensing_cmblikes",
    "bicep_keck_2015",
    "H0_riess2018a", "H0_riess2018b",
    "sdss_dr12_consensus_bao", "sdss_dr12_consensus_full_shape", "sdss_dr12_consensus_final",
    "sdss_dr7_mgs", "sixdf_2011_bao",
    "sn_jla", "sn_jla_lite", "sn_pantheon",
    "des_y1_shear", "des_y1_clustering", "des_y1_galaxy_galaxy"
]

# NB: It's possible that DES makes everything else slower, since it adds many more
# redshifts to the computation, so there are many more redshifts to select the
# required e.g. H(z) from, making e.g. SN and BAO likes slower (longer theory code request)

info[_likelihood] = odict([[l,None] for l in likelihoods])

# No fsigma8 in classy!
if list(info_theory)[0] == "classy":
    info[_likelihood].pop("sdss_dr12_consensus_final")
    info[_likelihood].pop("sdss_dr12_consensus_full_shape")

info[_timing] = True
info[_path_install] = modules
model = get_model(info)

In [ ]:
params_lowTEB_highTTTEEE.pop("H0", None)

for i in range(n):
    point = dict(zip(model.parameterization.sampled_params(), model.prior.sample(ignore_external=True)[0]))
    point.update(params_lowTEB_highTTTEEE)
    model.loglike(point, cached=False)

In [ ]:
from pandas import DataFrame
df = DataFrame(columns=["time", "speed", "old speed"], index=likelihoods)
for name in [_theory] + list(model.likelihood):
    like = model.likelihood[name]
    df.at[name, "time"] = like.time_avg
    df.at[name, "speed"] = 1/like.time_avg
    df.at[name, "old speed"] = like.speed
    df.at[name, "factor"] = like.speed*like.time_avg
print(df)